# Product-SKU Data Synthesis with Categories

This notebook implements the productId-SKU mapping generation as specified in task 02-productId-sku-generation.

## Overview
- Load products from actual scraped data: `src/data-extraction/data/products.csv` (including category and subcategory)
- Create DuckDB database with products table preserving all product information
- Generate product_skus mapping table where each productId has 1-3 randomly generated SKUs
- SKU format: `{productId}{4-digit-suffix}` where suffix is 0001, 0002, 0003

## Database Structure
1. **products table**: Contains all product information including:
   - productId (sku from CSV)
   - name, brandName, sellingSize
   - price (in pence), currency
   - category and subcategory (from API response)
   
2. **product_skus table**: Many-to-many mapping:
   - productId (references products.sku)
   - sku (warehouse SKU)
   - sku_suffix (0001, 0002, 0003)

## ⚠️ Important: Run cells in order
This notebook must be run sequentially from top to bottom. Each cell depends on variables and connections established in previous cells.

## 🔧 Troubleshooting
- If you get `NameError: name 'OUTPUT_PATH' is not defined`, run all cells from the beginning
- Use **Kernel → Restart & Run All** to ensure proper execution order
- Make sure you have installed dependencies: `~/miniconda3/envs/grocery_poc/bin/python -m pip install -r requirements.txt`
- Ensure you have run the data extraction scraper to generate `products.csv` with category data

## 0. Setup and Imports
**⚠️ This cell MUST be run first!**

In [19]:
# Standard library imports
import os
import random
from pathlib import Path

# Data manipulation and database
import pandas as pd
import duckdb

# Set random seed for reproducibility
random.seed(42)

# Print versions for debugging
print(f"Pandas version: {pd.__version__}")
print(f"DuckDB version: {duckdb.__version__}")

# Set up paths
PROJECT_ROOT = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
# Use actual scraped data from data-extraction
DATA_PATH = PROJECT_ROOT / "src" / "data-extraction" / "data" / "products.csv"
OUTPUT_PATH = PROJECT_ROOT / "docs" / "products" / "product-skus-mapping.csv"
DB_PATH = PROJECT_ROOT / "src" / "data" / "products.duckdb"

print(f"\nProject root: {PROJECT_ROOT}")
print(f"Data path: {DATA_PATH}")
print(f"Database will be created at: {DB_PATH}")

# Verify the data file exists
if DATA_PATH.exists():
    print(f"✅ Found scraped data file: {DATA_PATH}")
    print(f"   File size: {DATA_PATH.stat().st_size / 1024:.2f} KB")
else:
    print(f"❌ Data file not found at: {DATA_PATH}")
    print("   Please run the data extraction scraper first:")

Pandas version: 2.2.3
DuckDB version: 1.2.1

Project root: /home/abobreshov/work/dataart/talk2data/grocery_poc
Data path: /home/abobreshov/work/dataart/talk2data/grocery_poc/src/data-extraction/data/products.csv
Database will be created at: /home/abobreshov/work/dataart/talk2data/grocery_poc/src/data/products.duckdb
✅ Found scraped data file: /home/abobreshov/work/dataart/talk2data/grocery_poc/src/data-extraction/data/products.csv
   File size: 247.75 KB


### Database Connection Setup

In [20]:
# Create or connect to DuckDB database
conn = duckdb.connect(str(DB_PATH))
print(f"✅ DuckDB connection established at: {DB_PATH}")

✅ DuckDB connection established at: /home/abobreshov/work/dataart/talk2data/grocery_poc/src/data/products.duckdb


## 1. Load Products Data from CSV

In [21]:
# Load products data from CSV into DuckDB
# Using read_csv_auto for automatic type detection
conn.execute(f"""
    CREATE OR REPLACE TABLE products AS 
    SELECT * FROM read_csv_auto('{DATA_PATH}')
""")

# Display table schema
schema_df = conn.execute("DESCRIBE products").df()
print("✅ Products table created successfully from scraped data!")
print(f"\nTable schema ({len(schema_df)} columns):")
print(schema_df)

# Show first few rows to verify data
print("\n🔍 First 5 rows of scraped data:")
first_rows = conn.execute("SELECT * FROM products LIMIT 5").df()
print(first_rows)

✅ Products table created successfully from scraped data!

Table schema (8 columns):
   column_name column_type null   key default extra
0          sku     VARCHAR  YES  None    None  None
1         name     VARCHAR  YES  None    None  None
2    brandName     VARCHAR  YES  None    None  None
3  sellingSize     VARCHAR  YES  None    None  None
4     currency     VARCHAR  YES  None    None  None
5        price      BIGINT  YES  None    None  None
6     category     VARCHAR  YES  None    None  None
7  subcategory     VARCHAR  YES  None    None  None

🔍 First 5 rows of scraped data:
                  sku                                  name  brandName  \
0  000000000000546921             Slow Cooked BBQ Pork Ribs   OAKHURST   
1  000000000000384321            British Chicken Drumsticks  ASHFIELDS   
2  000000000000387288  Extra Large British Pork Loin Steaks  ASHFIELDS   
3  000000000000571453       Southern Fried Drums and Thighs   ROOSTERS   
4  000000000000571463                Sizzler 

### Data Validation and Overview

In [22]:
# Get basic statistics about the loaded data
product_count = conn.execute("SELECT COUNT(*) as count FROM products").fetchone()[0]
brand_count = conn.execute("SELECT COUNT(DISTINCT brandName) as count FROM products").fetchone()[0]
category_count = conn.execute("SELECT COUNT(DISTINCT category) as count FROM products WHERE category != ''").fetchone()[0]
subcategory_count = conn.execute("SELECT COUNT(DISTINCT subcategory) as count FROM products WHERE subcategory != ''").fetchone()[0]

print(f"📊 Data Summary:")
print(f"   - Total products: {product_count:,}")
print(f"   - Unique brands: {brand_count}")
print(f"   - Unique categories: {category_count}")
print(f"   - Unique subcategories: {subcategory_count}")
print(f"   - Price range: £{conn.execute('SELECT MIN(price)/100.0 FROM products').fetchone()[0]:.2f} - £{conn.execute('SELECT MAX(price)/100.0 FROM products').fetchone()[0]:.2f}")
print(f"\n🔍 Sample products (first 5):")

# Display sample with better formatting including categories
# Note: Price is in pence, so we divide by 100 to show in pounds
sample_df = conn.execute("""
    SELECT 
        sku as productId,
        name,
        brandName,
        category,
        subcategory,
        sellingSize,
        price as price_pence,
        ROUND(price/100.0, 2) as price_gbp
    FROM products 
    LIMIT 5
""").df()
sample_df

📊 Data Summary:
   - Total products: 2,501
   - Unique brands: 121
   - Unique categories: 5
   - Unique subcategories: 45
   - Price range: £0.16 - £13.49

🔍 Sample products (first 5):


,productId,name,brandName,category,subcategory,sellingSize,price_pence,price_gbp
0,000000000000546921,Slow Cooked BBQ Pork Ribs,OAKHURST,Fresh Food,Pork & Gammon,0.6 KG,469,4.69
1,000000000000384321,British Chicken Drumsticks,ASHFIELDS,Fresh Food,Poultry,2 KG,189,1.89
2,000000000000387288,Extra Large British Pork Loin Steaks,ASHFIELDS,Fresh Food,Pork & Gammon,1 KG,269,2.69
3,000000000000571453,Southern Fried Drums and Thighs,ROOSTERS,Fresh Food,Poultry,0.8 KG,399,3.99
4,000000000000571463,Sizzler Chicken Burger,ROOSTERS,Fresh Food,Poultry,0.3 KG,299,2.99


### Category Analysis

In [23]:
# Analyze product categories
print("📊 Category Distribution:")

category_df = conn.execute("""
    SELECT 
        category,
        COUNT(*) as product_count,
        COUNT(DISTINCT brandName) as brands,
        ROUND(AVG(price/100.0), 2) as avg_price_gbp,
        ROUND(MIN(price/100.0), 2) as min_price_gbp,
        ROUND(MAX(price/100.0), 2) as max_price_gbp
    FROM products
    WHERE category != ''
    GROUP BY category
    ORDER BY product_count DESC
    LIMIT 10
""").df()

print(f"\nTop 10 categories by product count:")
category_df

📊 Category Distribution:

Top 10 categories by product count:


,category,product_count,brands,avg_price_gbp,min_price_gbp,max_price_gbp
0,Food Cupboard,969,62,1.26,0.27,6.89
1,Chilled Food,817,48,1.93,0.39,5.59
2,Fresh Food,535,17,2.92,0.16,13.49
3,Bakery,154,20,1.39,0.45,7.49
4,Food,26,6,1.38,0.89,2.49


In [10]:
# Analyze subcategories within a specific category
print("📊 Subcategory Analysis (for products with subcategories):")

# First, let's see which categories have subcategories
subcategory_summary = conn.execute("""
    SELECT 
        category,
        subcategory,
        COUNT(*) as product_count,
        ROUND(AVG(price/100.0), 2) as avg_price_gbp
    FROM products
    WHERE subcategory != ''
    GROUP BY category, subcategory
    ORDER BY category, product_count DESC
    LIMIT 15
""").df()

print(f"\nTop subcategories by category:")
subcategory_summary

📊 Subcategory Analysis (for products with subcategories):

Top subcategories by category:


,category,subcategory,product_count,avg_price_gbp
0,Bakery,Cakes,48,1.75
1,Bakery,Bread,32,1.05
2,Bakery,Breakfast Pastries,21,1.37
3,Bakery,"Wraps, Naans, Pittas & Thins",16,1.25
4,Bakery,Bread Rolls,15,1.25
5,Bakery,Sweet Treats,9,1.21
6,Bakery,Bagels,3,1.09
7,Chilled Food,"Party Food, Pies & Salads",154,1.78
8,Chilled Food,Chilled Meats,111,1.86
9,Chilled Food,Cheese,101,2.06


## 2. Generate Product-SKU Mapping Table

Each productId (current SKU column) will have 1-3 randomly generated SKUs. The new SKUs will be created by appending a 4-digit suffix to the original productId.

In [11]:
# Get all unique productIds from the products table
print("🔄 Generating SKU mappings...")

product_ids = conn.execute("SELECT DISTINCT sku as productId FROM products ORDER BY sku").df()['productId'].tolist()

# Generate SKU mappings with controlled randomness
sku_mappings = []

for product_id in product_ids:
    # Randomly decide how many SKUs this product will have (1-3)
    num_skus = random.randint(1, 3)
    
    # Generate SKUs for this product by adding 4-digit suffix
    for i in range(num_skus):
        # Generate 4-digit suffix (0001, 0002, 0003)
        suffix = f"{i+1:04d}"
        sku = f"{product_id}{suffix}"
        
        sku_mappings.append({
            'productId': product_id,
            'sku': sku,
            'sku_suffix': suffix
        })

# Create DataFrame from mappings
sku_df = pd.DataFrame(sku_mappings)

print(f"✅ Generated {len(sku_mappings):,} SKU mappings for {len(product_ids):,} products")
print(f"   Average SKUs per product: {len(sku_mappings)/len(product_ids):.2f}")
print(f"\n📋 Sample SKU mappings (first 10):")
sku_df.head(10)

🔄 Generating SKU mappings...
✅ Generated 5,039 SKU mappings for 2,501 products
   Average SKUs per product: 2.01

📋 Sample SKU mappings (first 10):


,productId,sku,sku_suffix
0,000000000000198481,0000000000001984810001,0001
1,000000000000198481,0000000000001984810002,0002
2,000000000000198481,0000000000001984810003,0003
3,000000000000199825,0000000000001998250001,0001
4,000000000000201269,0000000000002012690001,0001
5,000000000000201731,0000000000002017310001,0001
6,000000000000201731,0000000000002017310002,0002
7,000000000000201731,0000000000002017310003,0003
8,000000000000201734,0000000000002017340001,0001
9,000000000000201734,0000000000002017340002,0002


### Store Mappings in Database

In [12]:
# Create product_skus table in DuckDB
print("💾 Creating product_skus table in database...")

# Drop existing table if it exists
conn.execute("DROP TABLE IF EXISTS product_skus")

# Create new table with proper schema
conn.execute("""
    CREATE TABLE product_skus (
        productId VARCHAR NOT NULL,
        sku VARCHAR NOT NULL PRIMARY KEY,
        sku_suffix VARCHAR(4) NOT NULL
    )
""")

# Insert data using parameterized query for safety
conn.executemany(
    "INSERT INTO product_skus VALUES (?, ?, ?)",
    [(row['productId'], row['sku'], row['sku_suffix']) for _, row in sku_df.iterrows()]
)

print("✅ product_skus table created and populated successfully!")

# Verify the data with summary statistics
summary_df = conn.execute("""
    SELECT 
        COUNT(DISTINCT productId) as unique_products,
        COUNT(*) as total_skus,
        ROUND(AVG(sku_count), 2) as avg_skus_per_product,
        MIN(sku_count) as min_skus_per_product,
        MAX(sku_count) as max_skus_per_product
    FROM (
        SELECT productId, COUNT(*) as sku_count 
        FROM product_skus 
        GROUP BY productId
    )
""").df()

print("\n📊 Table summary:")
summary_df

💾 Creating product_skus table in database...
✅ product_skus table created and populated successfully!

📊 Table summary:


,unique_products,total_skus,avg_skus_per_product,min_skus_per_product,max_skus_per_product
0,2501,2501,2.01,1,3


## 3. Analyze SKU Distribution

In [13]:
# Analyze the distribution of SKUs per product
print("📊 Distribution of SKUs per product:")

distribution_df = conn.execute("""
    SELECT 
        sku_count as skus_per_product,
        COUNT(*) as number_of_products,
        ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) as percentage,
        REPEAT('■', CAST(COUNT(*) / 10 AS INTEGER)) as visual_bar
    FROM (
        SELECT productId, COUNT(*) as sku_count 
        FROM product_skus 
        GROUP BY productId
    )
    GROUP BY sku_count
    ORDER BY sku_count
""").df()

distribution_df

📊 Distribution of SKUs per product:


,skus_per_product,number_of_products,percentage,visual_bar
0,1,805,32.19,■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■...
1,2,854,34.15,■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■...
2,3,842,33.67,■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■...


## 4. Example Queries and Usage

The following examples demonstrate how to work with the generated product-SKU mappings.

In [14]:
# Example 1: Get all SKUs for specific products with category info
print("🔍 Example 1: SKUs for first 3 unique products with category info\n")

example1_df = conn.execute("""
    WITH first_products AS (
        SELECT DISTINCT productId 
        FROM product_skus 
        ORDER BY productId 
        LIMIT 3
    )
    SELECT 
        p.sku as productId,
        p.name,
        p.brandName,
        p.category,
        p.subcategory,
        ps.sku,
        ps.sku_suffix
    FROM products p
    JOIN product_skus ps ON p.sku = ps.productId
    WHERE ps.productId IN (SELECT productId FROM first_products)
    ORDER BY p.sku, ps.sku_suffix
""").df()

example1_df

🔍 Example 1: SKUs for first 3 unique products with category info



,productId,name,brandName,category,subcategory,sku,sku_suffix
0,000000000000198481,Sunflower Oil,SOLESTA,Food Cupboard,"Sauces, Oils & Dressings",0000000000001984810001,0001
1,000000000000198481,Sunflower Oil,SOLESTA,Food Cupboard,"Sauces, Oils & Dressings",0000000000001984810002,0002
2,000000000000198481,Sunflower Oil,SOLESTA,Food Cupboard,"Sauces, Oils & Dressings",0000000000001984810003,0003
3,000000000000199825,Olive Oil Extra Virgin,SPECIALLY SELECTED,Food Cupboard,"Sauces, Oils & Dressings",0000000000001998250001,0001
4,000000000000201269,Pineapple Slices in Juice,FOUR SEASONS,Food Cupboard,Desserts,0000000000002012690001,0001


In [15]:
# Example 2: Products with maximum number of SKUs (3 SKUs)
print("🔍 Example 2: Products with 3 SKUs (showing first 5)\n")

example2_df = conn.execute("""
    SELECT 
        p.sku as productId,
        p.name,
        p.brandName,
        p.sellingSize,
        COUNT(ps.sku) as sku_count,
        STRING_AGG(ps.sku_suffix, ', ' ORDER BY ps.sku_suffix) as sku_suffixes
    FROM products p
    JOIN product_skus ps ON p.sku = ps.productId
    GROUP BY p.sku, p.name, p.brandName, p.sellingSize
    HAVING COUNT(ps.sku) = 3
    ORDER BY p.name
    LIMIT 5
""").df()

example2_df

🔍 Example 2: Products with 3 SKUs (showing first 5)



,productId,name,brandName,sellingSize,sku_count,sku_suffixes
0,000000000596657001,100% British Beef 30 Day Matured Aberdeen Angu...,SPECIALLY SELECTED,0.17 KG,3,"0001, 0002, 0003"
1,000000000000448938,100% British Beef Fillet Steaks 28 Day Matured...,ASHFIELDS,0.34 KG,3,"0001, 0002, 0003"
2,000000000600380001,100% British Beef Lean Aberdeen Angus Steak Mi...,SPECIALLY SELECTED,0.5 KG,3,"0001, 0002, 0003"
3,000000000402653002,100% British Beef Sliced Roasted Topside of Be...,SPECIALLY SELECTED,100 G,3,"0001, 0002, 0003"
4,000000000000466111,100% British Beef Wagyu Fillet Steak,SPECIALLY SELECTED,0.17 KG,3,"0001, 0002, 0003"


In [16]:
# Example 3: Brand-level SKU statistics
print("🔍 Example 3: SKU statistics by brand (top 10 brands by product count)\n")

brand_stats_df = conn.execute("""
    SELECT 
        p.brandName,
        COUNT(DISTINCT p.sku) as product_count,
        COUNT(ps.sku) as total_skus,
        ROUND(AVG(sku_per_product.sku_count), 2) as avg_skus_per_product
    FROM products p
    JOIN product_skus ps ON p.sku = ps.productId
    JOIN (
        SELECT productId, COUNT(*) as sku_count
        FROM product_skus
        GROUP BY productId
    ) sku_per_product ON p.sku = sku_per_product.productId
    GROUP BY p.brandName
    ORDER BY product_count DESC
    LIMIT 10
""").df()

brand_stats_df

🔍 Example 3: SKU statistics by brand (top 10 brands by product count)



,brandName,product_count,total_skus,avg_skus_per_product
0,SPECIALLY SELECTED,363,733,2.35
1,ASHFIELDS,160,315,2.33
2,NATURE'S PICK,151,307,2.39
3,THE DELI,111,227,2.37
4,BRAMWELLS,104,209,2.31
5,HARVEST MORN,96,199,2.38
6,SNACKRITE,82,166,2.34
7,"READY, SET…COOK!",76,159,2.42
8,BROOKLEA,75,160,2.40
9,EVERYDAY ESSENTIALS,70,132,2.23


In [17]:
# Example 4: Find products by category
print("🔍 Example 4: Products in a specific category with their SKUs\n")

# Let's find products in the 'Chilled Food' category
example4_df = conn.execute("""
    SELECT 
        p.sku as productId,
        p.name,
        p.brandName,
        p.category,
        p.subcategory,
        p.price as price_pence,
        ROUND(p.price/100.0, 2) as price_gbp,
        COUNT(ps.sku) as num_skus,
        STRING_AGG(ps.sku, ', ' ORDER BY ps.sku) as skus
    FROM products p
    JOIN product_skus ps ON p.sku = ps.productId
    WHERE p.category = 'Chilled Food'
    GROUP BY p.sku, p.name, p.brandName, p.category, p.subcategory, p.price
    ORDER BY p.subcategory, p.name
    LIMIT 10
""").df()

print("Products in 'Chilled Food' category:")
example4_df

🔍 Example 4: Products in a specific category with their SKUs

Products in 'Chilled Food' category:


,productId,name,brandName,category,subcategory,price_pence,price_gbp,num_skus,skus
0,000000000282692004,12 Months Matured Comté Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,319,3.19,3,"0000000002826920040001, 0000000002826920040002..."
1,000000000283099005,Baked Camembert French Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,259,2.59,3,"0000000002830990050001, 0000000002830990050002..."
2,000000000552214002,Barber's Cave Aged Extra Mature British Chedda...,SPECIALLY SELECTED,Chilled Food,Cheese,229,2.29,2,"0000000005522140020001, 0000000005522140020002"
3,000000000635391002,Barbers Mature Cheddar Cheese Bake,SPECIALLY SELECTED,Chilled Food,Cheese,299,2.99,2,"0000000006353910020001, 0000000006353910020002"
4,000000000000558048,Barrel Aged Feta Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,289,2.89,3,"0000000000005580480001, 0000000000005580480002..."
5,000000000423865006,Beacon Blue British Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,269,2.69,2,"0000000004238650060001, 0000000004238650060002"
6,000000000417200002,Belton Farm Crumbly Wensleydale Cheese,EMPORIUM,Chilled Food,Cheese,189,1.89,2,"0000000004172000020001, 0000000004172000020002"
7,000000000552214001,Belton Farm Red Fox Vintage Red Leicester Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,229,2.29,1,0000000005522140010001
8,000000000304786004,Black Pepper & Paprika Mini Roule Cheese,SPECIALLY SELECTED,Chilled Food,Cheese,135,1.35,2,"0000000003047860040001, 0000000003047860040002"
9,000000000423342001,Black Pepper Flavoured Cheddar Cheese Slices 5...,EMPORIUM,Chilled Food,Cheese,179,1.79,1,0000000004233420010001


## 5. Export Tables for Future Use

⚠️ **Note**: This cell requires variables from earlier cells. If you get a `NameError`, please run all cells from the beginning.

In [18]:
## Summary

This notebook successfully:
1. ✅ Loaded products from the actual scraped data (`src/data-extraction/data/products.csv`) with category and subcategory information
2. ✅ Created a DuckDB database with products table including category hierarchy
3. ✅ Generated unique SKUs by appending 4-digit suffixes (0001-0003) 
4. ✅ Created a product_skus mapping table with 1-3 SKUs per product (many-to-many relationship)
5. ✅ Analyzed category distribution and subcategory relationships
6. ✅ Stored all data in DuckDB for efficient querying
7. ✅ Exported the mappings to CSV for external use

The database now contains:
- **products table**: Complete product information including categories and subcategories
- **product_skus table**: Many-to-many mapping between productIds and warehouse SKUs

The generated data is now ready for use in the grocery POC system with full category support.

SyntaxError: invalid character '✅' (U+2705) (4122010041.py, line 4)

## Summary

This notebook successfully:
1. ✅ Loaded products from the actual scraped data (`src/data-extraction/data/products.csv`)
2. ✅ Generated unique SKUs by appending 4-digit suffixes (0001-0003)
3. ✅ Created a mapping table with 1-3 SKUs per product
4. ✅ Stored the data in DuckDB for efficient querying
5. ✅ Exported the mappings to CSV for external use

The generated data is now ready for use in the grocery POC system.